In [ ]:
import numpy as np
import copy
from collections import namedtuple
from matplotlib import pyplot as plt

In [ ]:
test_text = """###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############"""

In [ ]:
with open("input.txt") as f:
    data_text = f.read()

In [ ]:
def add_tuple(a, b):
    return(a[0] + b[0], a[1] + b[1])

In [ ]:
class Maze:
    def __init__(self, maze: np.ndarray):
        self.shape = maze.shape
        self.pos = tuple(map(int, np.where(maze == "S")))
        self.dir = (0, 1)
        self.end = tuple(map(int, np.where(maze == "E")))
        self.visited = []
        self.turns = []
        self.score = 0
        self.stuck = False
    
    def _get_score(self, dp: tuple, dir: tuple):
        if dp == dir:
            return 1
        
        return 1001
    
    def possible_moves(self, walls):
        dps = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        found_possible_move = False
        new_ways = []
        starting_pos = self.pos
        starting_dir = self.dir
        starting_score = self.score
        starting_turns = copy.copy(self.turns)
        for dp in dps:
            candidate = add_tuple(starting_pos, dp)
            if candidate not in walls and candidate not in self.visited:
                if not found_possible_move:
                    self.visited.append(self.pos)
                    if starting_dir != dp:
                        self.turns.append(self.pos)
                    self.pos = candidate
                    self.score += self._get_score(dp, self.dir)
                    self.dir = dp
                    found_possible_move = True
                else:
                    new_maze = copy.deepcopy(self)
                    new_maze.turns = copy.copy(starting_turns)
                    if starting_dir != dp:
                        new_maze.turns.append(starting_pos)
                    new_maze.pos = candidate
                    new_maze.dir = dp
                    new_maze.score = starting_score + new_maze._get_score(dp, starting_dir)
                    new_ways.append(new_maze)
        if not found_possible_move:
            self.stuck = True
        return new_ways

    def __bool__(self):
        if self.pos == self.end:
            return True
        return False
    
    def update(self, maze):
        new_maze = np.full(maze.shape, ".")
        for pos in self.walls:
            new_maze[pos] = "#"
        new_maze[self.end] = "E"
 
        for pos in self.visited:
            new_maze[pos] = "i"
        for pos in self.turns:
            new_maze[pos] = "T"
        new_maze[self.pos] = "R"
        return new_maze
        
    def to_img(self, walls):
        img = np.full((self.shape[0], self.shape[1], 3), 0, dtype=np.uint8)
        for pos in walls:
            img[pos] = [1, 1, 1]
        img[self.end] = [0, 1, 0]
 
        for pos in self.visited:
            img[pos] = [0, 0, 1]
        for pos in self.turns:
            img[pos] = [1, 0, 1]
        img[self.pos] = [1, 0, 0]
        img *= 255
        return img

In [ ]:
def parse_input(text: str):
    maze_arr = np.array([[s for s in line] for line in text.strip().split("\n")])
    return maze_arr, Maze(maze_arr)

### Part one

In [ ]:

maze_arr, starting_maze = parse_input(data_text)
walls = [(int(x), int(y)) for x, y in zip(*np.where(maze_arr == "#"))]
possible_points = []
mazes = [starting_maze]
itr = 0
visited = np.full_like(maze_arr, 10000000000000, dtype=int)
while True:
    itr += 1
    if visited[mazes[0].pos] + 1000 < mazes[0].score:
        mazes.pop(0)
        continue
    else:
        visited[mazes[0].pos] = mazes[0].score 
    new_ways = mazes[0].possible_moves(walls)
    if new_ways:
        mazes += new_ways
    if any(new_ways) or mazes[0]:
        for i in range(1, len(mazes)):
            new_ways = mazes[i].possible_moves(walls)
            mazes += new_ways
        break
    if mazes[0].stuck:
        mazes.pop(0)
    if len(mazes) == 1:
        continue
    min_maze = min(mazes, key=lambda x: x.score)
    if min_maze.score < mazes[0].score:
        min_id = mazes.index(min_maze)
        mazes[0], mazes[min_id] = mazes[min_id], mazes[0]
print(mazes[0].score)        

### Part two

In [ ]:
good_mazes = [maze for maze in mazes if (maze.score <= mazes[0].score and maze.pos == maze.end)]

In [ ]:
intersection = copy.deepcopy(good_mazes[0].visited)
for i in range(1, len(good_mazes)):
    intersection += [pos for pos in good_mazes[i].visited if pos not in intersection]

test_arr = np.zeros_like(maze_arr, dtype=int)
for pos in intersection:
    test_arr[pos] = 1
print(test_arr.sum() + 1)